In [ ]:
import datetime, requests
import pandas as pd
from pandas import Timestamp
import xlsxwriter

In [ ]:
codigoINE_empresas = {
    "Constituidas" : 13917,
    "Aumento de Capital" : 13922,
    "Disueltas" : 13923
}

hours_added = datetime.timedelta(hours = 1)
num_datos = 50000
url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'

In [62]:
df_empresas = pd.DataFrame(columns=[])

for tipo in codigoINE_empresas:
    print(tipo)
    main_url_INE_Estructura_Datos = "https://servicios.ine.es/wstempus/js/ES/SERIES_TABLA/" + str(codigoINE_empresas[tipo]) + "?page=1"
    INE_Estructura_Datos = requests.get(main_url_INE_Estructura_Datos).json()

    df_INE_Estructura_Datos_Empresas = pd.DataFrame.from_dict(INE_Estructura_Datos)
    
    df_INE_Estructura_Datos_Empresas["Nombre"] = df_INE_Estructura_Datos_Empresas["Nombre"].str.replace("S. Comanditarias y S. Colectivas", "Sociedades Comanditarias y Sociedades Colectivas")
    df_INE_Estructura_Datos_Empresas["Nombre"] = df_INE_Estructura_Datos_Empresas["Nombre"].str.strip()
    
    new = df_INE_Estructura_Datos_Empresas["Nombre"].str.split('.')
    if tipo == "Disueltas":
        df_INE_Estructura_Datos_Empresas["Nombre"] = new.map(lambda x: x[0])
        df_INE_Estructura_Datos_Empresas["Actividad"] = new.map(lambda x: x[2])
        df_INE_Estructura_Datos_Empresas["Tipo Empresa"] = new.map(lambda x: x[1])
        df_INE_Estructura_Datos_Empresas["Dato"] = new.map(lambda x: x[3])
    else :
        df_INE_Estructura_Datos_Empresas["Nombre"] = new.map(lambda x: x[0])
        df_INE_Estructura_Datos_Empresas["Actividad"] = new.map(lambda x: x[1])
        df_INE_Estructura_Datos_Empresas["Tipo Empresa"] = new.map(lambda x: x[2])
        df_INE_Estructura_Datos_Empresas["Dato"] = new.map(lambda x: x[3])

    df_INE_Estructura_Datos_Empresas.drop(df_INE_Estructura_Datos_Empresas[df_INE_Estructura_Datos_Empresas.Nombre == "Sociedades Constituídas"].index, inplace=True)
    df_INE_Estructura_Datos_Empresas.drop(df_INE_Estructura_Datos_Empresas[df_INE_Estructura_Datos_Empresas.Nombre == "Aumento de Capital"].index, inplace=True)
    df_INE_Estructura_Datos_Empresas.drop(df_INE_Estructura_Datos_Empresas[df_INE_Estructura_Datos_Empresas.Nombre == "Total"].index, inplace=True)
    df_INE_Estructura_Datos_Empresas.drop(df_INE_Estructura_Datos_Empresas[df_INE_Estructura_Datos_Empresas.Nombre == "Total Nacional"].index, inplace=True)
    df_INE_Estructura_Datos_Empresas.drop(df_INE_Estructura_Datos_Empresas[df_INE_Estructura_Datos_Empresas.Nombre == "Voluntaria"].index, inplace=True)
    df_INE_Estructura_Datos_Empresas.drop(df_INE_Estructura_Datos_Empresas[df_INE_Estructura_Datos_Empresas.Nombre == "Por fusión"].index, inplace=True)
    df_INE_Estructura_Datos_Empresas.drop(df_INE_Estructura_Datos_Empresas[df_INE_Estructura_Datos_Empresas.Nombre == "Otras"].index, inplace=True)
    
    df_INE_Estructura_Datos_Empresas = df_INE_Estructura_Datos_Empresas[["Id","COD","Nombre","Actividad","Tipo Empresa", "Dato"]]
    #display(df_INE_Estructura_Datos_Empresas)

    for index, provincia in df_INE_Estructura_Datos_Empresas.iterrows():
        #print(provincia["Nombre"])
        codigo = provincia["COD"]
        url_datos = url_plantilla.format(codigo=codigo, num_datos=num_datos)
        #display(pd.DataFrame.from_dict(requests.get(url_datos).json()["Data"]))
        df_empresas_ = pd.DataFrame.from_dict(requests.get(url_datos).json()["Data"])[["Anyo","Fecha","Valor"]].rename(columns={"Anyo": "Año", "Fecha": "Fecha", "Valor": provincia["Dato"].strip()})
        df_empresas_["Fecha"] = pd.to_datetime(df_empresas_["Fecha"], unit='ms')
        df_empresas_["Fecha"] = df_empresas_["Fecha"].map(lambda x: x + hours_added)
        df_empresas_['Mes'] = df_empresas_['Fecha'].dt.month
        df_empresas_ = df_empresas_[["Año", "Mes", provincia["Dato"].strip()]]
        df_empresas_ = df_empresas_.set_index(["Año", "Mes"])
        
        #display(df_empresas_)
        
        df_empresas_.columns = pd.MultiIndex.from_product([[provincia["Nombre"].strip()], [provincia["Actividad"].strip()], [provincia["Tipo Empresa"].strip()], df_empresas_.columns], names=["Provincia", "Actividad", "Tipo de Empresa", "Dato"])
        
        #display(df_empresas_)
                                    
        df_empresas = pd.concat([df_empresas, df_empresas_], axis=1)
        #display(df_empresas)


Constituidas
Aumento de Capital
Disueltas


In [88]:
display(df_empresas)

Provincia                      Albacete        Alicante/Alacant  \
Actividad       Sociedades Constituídas Sociedades Constituídas   
Tipo de Empresa             Mercantiles             Mercantiles   
Dato                          Dato base               Dato base   
Año  Mes                                                          
2008 1                            109.0                   580.0   
     2                            804.0                   499.0   
     3                            103.0                   396.0   
     3                            119.0                   562.0   
     4                            125.0                   448.0   
...                                 ...                     ...   
2020 9                             45.0                   360.0   
     11                            63.0                   390.0   
     12                            49.0                   350.0   
2021 1                             49.0                   400.0   
     2                             51.0                   440.0   

Provincia                       Almería             Araba/Álava  \
Actividad       Sociedades Constituídas Sociedades Constituídas   
Tipo de Empresa             Mercantiles             Mercantiles   
Dato                          Dato base               Dato base   
Año  Mes                                                          
2008 1                            157.0                    93.0   
     2                            129.0                    51.0   
     3                            163.0                    54.0   
     3                            220.0                    70.0   
     4                            113.0                    75.0   
...                                 ...                     ...   
2020 9                            125.0                    36.0   
     11                           136.0                    34.0   
     12                           126.0                    29.0   
2021 1                            132.0                    27.0   
     2                            157.0                    38.0   

Provincia                      Asturias                   Ávila  \
Actividad       Sociedades Constituídas Sociedades Constituídas   
Tipo de Empresa             Mercantiles             Mercantiles   
Dato                          Dato base               Dato base   
Año  Mes                                                          
2008 1                            172.0                    26.0   
     2                            173.0                    35.0   
     3                            157.0                    58.0   
     3                            168.0                    24.0   
     4                            147.0                    20.0   
...                                 ...                     ...   
2020 9                            103.0                     5.0   
     11                            95.0                    13.0   
     12                           103.0                    10.0   
2021 1                            115.0                    14.0   
     2                            124.0                     5.0   

Provincia                       Badajoz          Balears, Illes  \
Actividad       Sociedades Constituídas Sociedades Constituídas   
Tipo de Empresa             Mercantiles             Mercantiles   
Dato                          Dato base               Dato base   
Año  Mes                                                          
2008 1                            142.0                   356.0   
     2                            129.0                   385.0   
     3                             98.0                   275.0   
     3                            135.0                   299.0   
     4                            146.0                   318.0   
...                                 ...                     ...   
2020 9                             76.0     

In [90]:
provincias = list(df_empresas.columns.levels[0])
df_empresas = df_empresas.groupby(level = 0, axis = 1)

df_situacion_empresas = pd.DataFrame(columns=[])

for pro in provincias:
    df_empresas2 = df_empresas.get_group(pro)
    df_situacion_empresas = pd.concat([df_situacion_empresas, df_empresas2], axis=1)

#display(df_situacion_empresas)


In [133]:
display(df_situacion_empresas["Bizkaia"]["Sociedades Constituídas"]["Mercantiles"].loc[(2021,1)])
display(df_situacion_empresas[df_situacion_empresas.index.get_level_values("Mes") == 1]["Bizkaia"]["Sociedades Constituídas"]["Mercantiles"])


,Dato,Dato base,Capital
Año,Mes,,
2021,1,145.0,13333.0


,Dato,Dato base,Capital
Año,Mes,,
2008,1,248.0,5786.0
2009,1,144.0,11128.0
2010,1,158.0,8136.0
2011,1,178.0,11985.0
2012,1,187.0,6377.0
2013,1,183.0,78111.0
2014,1,194.0,3852.0
2015,1,139.0,10135.0
2016,1,119.0,7388.0


In [ ]:
#df_situacion_empresas.to_excel("output.xlsx")